In [12]:
import os
import cv2
from src import *
from tqdm import tqdm
import math

destination = "./patch_dataset"
image_root = os.path.join(os.environ["DATASET_ROOT"], "bcd2022", "images_1024")

In [19]:
def get_mil_instance_grid(image, num_wh, overlap_perc_wh, input_size=None):
    img_h, img_w = image.shape
    instance_w = math.floor(img_w / ((1 - overlap_perc_wh[0]) * (num_wh[0] - 1) + 1))
    instance_h = math.floor(img_h / ((1 - overlap_perc_wh[1]) * (num_wh[1] - 1) + 1))

    step_w = (1 - overlap_perc_wh[0]) * instance_w
    step_h = (1 - overlap_perc_wh[1]) * instance_h

    instance_list = []
    for i in range(num_wh[1]):
        i1 = int(i * step_h)
        i2 = int(i1 + instance_h)
        for j in range(num_wh[0]):
            j1 = int(j * step_w)
            j2 = int(j1 + instance_w)
            if input_size is not None:
                instance = cv2.resize(image[i1:i2, j1:j2], input_size)
            else:
                instance = image[i1:i2, j1:j2]

            instance_list.append(instance)

    return instance_list

In [31]:
files = os.listdir(image_root)
nfile = len(files)

os.makedirs(destination, exist_ok=True)

index = 0
for k, f in enumerate(tqdm(files)):
    path = os.path.join(image_root, f)
    img = cv2.imread(path, 0)
    img = crop_breast(img)
    instances = get_mil_instance_grid(img, (4, 4), (0.1, 0.1), (256, 256))
    for patch in instances:
        if patch.mean() > 15:
            cv2.imwrite(os.path.join(destination, f"{index:06d}.png"), patch)
        index += 1
    break

  0%|                                                 | 0/54706 [00:00<?, ?it/s]

0.0
11.033401489257812
45.13029479980469
81.68072509765625
39.03852844238281
117.86419677734375
125.6414794921875
116.24095153808594
39.945648193359375
107.24966430664062
117.49722290039062
116.74609375
0.0
12.459320068359375
41.148101806640625
66.90184020996094
